# Lecture 2: Introduction to Feature Engineering

## Setup

In [1]:
import numpy as np 
import pandas as pd
import random
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import preprocessing

In [2]:
articles = pd.read_csv('articles.csv.zip')
customers = pd.read_csv('customers.csv.zip')
sample_submisison = pd.read_csv('sample_submission.csv.zip')
transactions = pd.read_csv('transactions_train.csv.zip')

## The H&M Dataset

In [3]:
articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105542 entries, 0 to 105541
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   article_id                    105542 non-null  int64 
 1   product_code                  105542 non-null  int64 
 2   prod_name                     105542 non-null  object
 3   product_type_no               105542 non-null  int64 
 4   product_type_name             105542 non-null  object
 5   product_group_name            105542 non-null  object
 6   graphical_appearance_no       105542 non-null  int64 
 7   graphical_appearance_name     105542 non-null  object
 8   colour_group_code             105542 non-null  int64 
 9   colour_group_name             105542 non-null  object
 10  perceived_colour_value_id     105542 non-null  int64 
 11  perceived_colour_value_name   105542 non-null  object
 12  perceived_colour_master_id    105542 non-null  int64 
 13 

In [4]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1371980 entries, 0 to 1371979
Data columns (total 7 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   customer_id             1371980 non-null  object 
 1   FN                      476930 non-null   float64
 2   Active                  464404 non-null   float64
 3   club_member_status      1365918 non-null  object 
 4   fashion_news_frequency  1355971 non-null  object 
 5   age                     1356119 non-null  float64
 6   postal_code             1371980 non-null  object 
dtypes: float64(3), object(4)
memory usage: 73.3+ MB


In [5]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   t_dat             object 
 1   customer_id       object 
 2   article_id        int64  
 3   price             float64
 4   sales_channel_id  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 1.2+ GB


In [6]:
# X = transactions.merge(customers, how='inner', on='customer_id')
# X = X.merge(articles, how='inner', on='article_id')

### Creating Samples 
If you would rather work with samples instead of the whole dataset (while prototyping your code). You can use the code below:

In [7]:
# Adapted from: https://www.kaggle.com/code/paweljankiewicz/hm-create-dataset-samples
# This extracts three sampled datasets, containing 0.1%, 1% and 5% of all users and their transactions, and the associated articles.
for sample_repr, sample in [("01", 0.001), ("1", 0.01), ("5", 0.05)]:
    print(sample)
    customers_sample = customers.sample(int(customers.shape[0]*sample), replace=False)
    customers_sample_ids = set(customers_sample["customer_id"])
    transactions_sample = transactions[transactions["customer_id"].isin(customers_sample_ids)]
    articles_sample_ids = set(transactions_sample["article_id"])
    articles_sample = articles[articles["article_id"].isin(articles_sample_ids)]
    customers_sample.to_csv(f"customers_sample{sample_repr}.csv.gz", index=False)
    transactions_sample.to_csv(f"transactions_sample{sample_repr}.csv.gz", index=False)
    articles_sample.to_csv(f"articles_sample{sample_repr}.csv.gz", index=False)

0.001
0.01
0.05


In [8]:
articles_sample = pd.read_csv('articles_sample5.csv.gz')
customers_sample = pd.read_csv('customers_sample5.csv.gz')
transactions_sample = pd.read_csv('transactions_sample5.csv.gz')

In [9]:
customers_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68599 entries, 0 to 68598
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   customer_id             68599 non-null  object 
 1   FN                      24001 non-null  float64
 2   Active                  23327 non-null  float64
 3   club_member_status      68289 non-null  object 
 4   fashion_news_frequency  67778 non-null  object 
 5   age                     67868 non-null  float64
 6   postal_code             68599 non-null  object 
dtypes: float64(3), object(4)
memory usage: 3.7+ MB


In [10]:
transactions_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1592642 entries, 0 to 1592641
Data columns (total 5 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   t_dat             1592642 non-null  object 
 1   customer_id       1592642 non-null  object 
 2   article_id        1592642 non-null  int64  
 3   price             1592642 non-null  float64
 4   sales_channel_id  1592642 non-null  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 60.8+ MB


## A Simplified Task: Binary Classification

The task of predicting which 12 items users are most likely to buy in the next week is difficult to translate to a traditional classification machine learning setting. 
To obtain the 12 items a user is most likely to buy, we need to make predictions for all items (or the ones selected by a baseline) and select the 12 that have the highest predicted scores.

In this assignment, we'll consider a simplified task: Predict whether a user ordered a single item or not, based on the features of the user and the item. 
We provide a baseline logistic regression model below, but haven't done much feature preprocessing or engineering!
Initially, it is always best to focus your efforts on getting your features in the right shape and setting up the right validation scheme and baselines.
Once you are sure that your features add value and your validation scheme is correct, then you typically move on to trying more elaborate models.

### Creating the Dataset

In [11]:
# If you'd rather use a sample. Uncomment the following code:
transactions = transactions_sample
customers = customers_sample
articles = articles_sample

In [12]:
transactions['ordered'] = 1

The problem setting is an example of a "PU learning" problem, i.e. only positives are labeled, everything else is unlabeled (and can be either positive or negative). 
Of course, we cannot train a classifier with just positive samples: The classifier will just learn that everything is positive.
Therefore, we need to manually generate negative samples.

Below, we use a simple random negative sampling strategy.
We want to create a balanced dataset, meaning that we have just as many positives as negatives.
This makes sure that the classifier will not benefit from predicting the positive/negative class more often than the other.
Realistically, the amount of positive samples is of course many times smaller than the amount of unlabeled, possibly negative instances.


If you want to try your hand at a more complex negative sampling strategy, you may want to check out this blog as a starting point: https://medium.com/mlearning-ai/overview-negative-sampling-on-recommendation-systems-230a051c6cd7.



In [13]:
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id,ordered
0,2018-09-20,0045c79125b4dc958579f902b49eacd8598f9eeaa12205...,211143037,0.033881,2,1
1,2018-09-20,0045c79125b4dc958579f902b49eacd8598f9eeaa12205...,211143037,0.033881,2,1
2,2018-09-20,0045c79125b4dc958579f902b49eacd8598f9eeaa12205...,503729006,0.042356,2,1
3,2018-09-20,0045c79125b4dc958579f902b49eacd8598f9eeaa12205...,503729006,0.042356,2,1
4,2018-09-20,00b6ec8613e51d8eadc5157f5a12ae1366ca29da7d44cc...,529008008,0.013542,2,1


In [14]:
# What's happening here? 
# We're creating negative samples. I.e. we're creating transactions that didn't actually occur.
# First, we need to know which interactions did occur:
positive_pairs = list(map(tuple, transactions[['customer_id', 'article_id']].drop_duplicates().values))

In [15]:
# Then we need to know what every synthetic transaction should contain: a date, a customer_id, an article_id, price, sales_channel_id. We will set ordered = 0, as these transactions didn't really occur.
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id,ordered
0,2018-09-20,0045c79125b4dc958579f902b49eacd8598f9eeaa12205...,211143037,0.033881,2,1
1,2018-09-20,0045c79125b4dc958579f902b49eacd8598f9eeaa12205...,211143037,0.033881,2,1
2,2018-09-20,0045c79125b4dc958579f902b49eacd8598f9eeaa12205...,503729006,0.042356,2,1
3,2018-09-20,0045c79125b4dc958579f902b49eacd8598f9eeaa12205...,503729006,0.042356,2,1
4,2018-09-20,00b6ec8613e51d8eadc5157f5a12ae1366ca29da7d44cc...,529008008,0.013542,2,1


In [16]:
# Extract real values
real_dates = transactions["t_dat"].unique()
real_customers = transactions["customer_id"].unique()
real_articles = transactions["article_id"].unique()
real_channels = transactions["sales_channel_id"].unique()
article_and_price = transactions[["article_id","price"]].drop_duplicates("article_id").set_index("article_id").squeeze()

In [17]:
# How many negatives do we need to sample?
num_neg_pos = transactions.shape[0]
print(num_neg_pos)

1592642


In [18]:
# Sampling negatives by selecting random users, articles, dates and sales channel:
# Note: This is quite naive. Some articles may not even have been available at the date we are sampling.
random.seed(42)

# Afterwards, we need to remove potential duplicates, so we'll sample too many.
num_neg_samples = int(num_neg_pos * 1.1)

# Sample each of the independent attributes.
neg_dates = np.random.choice(real_dates, size=num_neg_samples)
neg_articles = np.random.choice(real_articles, size=num_neg_samples)
neg_customers = np.random.choice(real_customers, size=num_neg_samples)
neg_channels = np.random.choice(real_channels, size=num_neg_samples)
ordered = np.array([0] * num_neg_samples)
# Assign to every article a real price.
neg_prices = article_and_price[neg_articles].values

In [19]:
neg_transactions = pd.DataFrame([neg_dates, neg_customers, neg_articles, neg_prices, neg_channels, ordered], index=transactions.columns).T

In [20]:
# Result:
neg_transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id,ordered
0,2019-03-02,aa91dfce559559049f83c35e6cabdfe2c95f66922e65cd...,678942054,0.016932,2,0
1,2018-12-18,55f0458b57e61409ba79ac437e15c53029250c88be4ddd...,579541086,0.016932,2,0
2,2019-06-16,2072516643058fb399f98e5216d290469eb04c3e5d3ee6...,692653004,0.06778,2,0
3,2018-10-09,193eb89961d94ccb407e33eef20b812907abbcac105066...,810671003,0.033881,1,0
4,2019-12-09,63205154139b0f219c31ed600d3c280d4cc049278d3c3d...,564786021,0.018712,2,0


In [21]:
neg_transactions.shape

(1751906, 6)

In [22]:
# Remove random negative samples that actually coincide with positives
df = neg_transactions[
    ~neg_transactions.set_index(["customer_id", "article_id"]).index.isin(positive_pairs)
]

# Remove any excess
chosen_neg_transactions = df.sample(num_neg_pos)

In [23]:
# Concat the negative samples to the positive samples:
transactions = pd.concat([transactions, chosen_neg_transactions])
transactions = transactions.merge(customers, how="inner", on='customer_id')
transactions = transactions.merge(articles, how="inner", on='article_id')

In [24]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3185284 entries, 0 to 3185283
Data columns (total 36 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   t_dat                         object 
 1   customer_id                   object 
 2   article_id                    object 
 3   price                         object 
 4   sales_channel_id              object 
 5   ordered                       object 
 6   FN                            float64
 7   Active                        float64
 8   club_member_status            object 
 9   fashion_news_frequency        object 
 10  age                           float64
 11  postal_code                   object 
 12  product_code                  int64  
 13  prod_name                     object 
 14  product_type_no               int64  
 15  product_type_name             object 
 16  product_group_name            object 
 17  graphical_appearance_no       int64  
 18  graphical_appearance_n

### Basic Preprocessing
Some very basic preprocessing.

In [25]:
# I'm dropping a lot of columns, use them in your engineering tasks!
transactions_processed = transactions[['customer_id', 'age', 'article_id', 'sales_channel_id', 'price', 'ordered']].copy()
transactions_processed.head()

,customer_id,age,article_id,sales_channel_id,price,ordered
0,0045c79125b4dc958579f902b49eacd8598f9eeaa12205...,55.0,211143037,2,0.033881,1
1,0045c79125b4dc958579f902b49eacd8598f9eeaa12205...,55.0,211143037,2,0.033881,1
2,c6a3fc64f3c483dab52f69ceabd3c71da48583d5a0160b...,24.0,211143037,2,0.033881,1
3,53f595ec12b425f4bcc972b83954bba9166ff46de4dd9c...,57.0,211143037,2,0.033881,1
4,6a430fdc951a5431e3499c724d5e176914a58c79323383...,22.0,211143037,2,0.033881,1


In [26]:
# Does it make sense to label encode?
# Label encoding the customer and article IDs:
customer_encoder = preprocessing.LabelEncoder()
article_encoder = preprocessing.LabelEncoder()

In [27]:
transactions_processed['customer_id'] = customer_encoder.fit_transform(transactions_processed['customer_id'])
transactions_processed['article_id'] = article_encoder.fit_transform(transactions_processed['article_id'])

In [28]:
# If you want to go back to the original encoding:
customer_encoder.inverse_transform([2])

array(['000412345109c7c085b5faec96afe864b19a172fa4cb9b94823d9ef6a8a1846a'],
      dtype=object)

In [29]:
transactions_processed.head()

,customer_id,age,article_id,sales_channel_id,price,ordered
0,74,55.0,188,2,0.033881,1
1,74,55.0,188,2,0.033881,1
2,53025,24.0,188,2,0.033881,1
3,22507,57.0,188,2,0.033881,1
4,28499,22.0,188,2,0.033881,1


In [30]:
# Can you come up with a NaN strategy that makes sense for each column in the dataset?
# Imputing all NaN values with zeros:
transactions_processed = transactions_processed.fillna(0)
transactions_processed.isnull().values.any()

False

In [31]:
# Does it make sense to one-hot encode?
# One-hot-encoding sales_channel_id:
transactions_processed = pd.get_dummies(transactions_processed, columns=['sales_channel_id'])

In [32]:
transactions_processed.head()

,customer_id,age,article_id,price,ordered,sales_channel_id_1,sales_channel_id_2
0,74,55.0,188,0.033881,1,0,1
1,74,55.0,188,0.033881,1,0,1
2,53025,24.0,188,0.033881,1,0,1
3,22507,57.0,188,0.033881,1,0,1
4,28499,22.0,188,0.033881,1,0,1


In [33]:
# Creating a Train / Test Split:
X_train, X_test, y_train, y_test = train_test_split(transactions_processed.drop('ordered', axis=1), transactions_processed['ordered'], test_size=0.10, random_state=42)

In [34]:
X_train.head()

,customer_id,age,article_id,price,sales_channel_id_1,sales_channel_id_2
2192727,4901,47.0,42121,0.040661,1,0
1671037,48582,55.0,68526,0.010153,1,0
2121851,47461,50.0,28336,0.020322,0,1
1330647,34317,38.0,15269,0.008458,1,0
305137,2463,18.0,38933,0.008458,0,1


In [35]:
y_train.head()

2192727    0
1671037    0
2121851    0
1330647    1
305137     1
Name: ordered, dtype: int64

## Baseline Model

In [36]:
# Will take a few minutes to run, if you're using the whole dataset:
baseline = LogisticRegression(random_state=42)
baseline = baseline.fit(X_train, y_train)

In [37]:
baseline.predict_proba(X_test)

array([[0.51655329, 0.48344671],
       [0.4984051 , 0.5015949 ],
       [0.50039638, 0.49960362],
       ...,
       [0.46900215, 0.53099785],
       [0.50401284, 0.49598716],
       [0.50033707, 0.49966293]])

In [38]:
y_test

2818105    1
93830      1
3148046    0
2710665    0
585371     1
          ..
595318     1
388792     0
2310528    0
1302354    0
2687735    0
Name: ordered, Length: 318529, dtype: int64

In [39]:
# Mean Accuracy:
baseline.score(X_test, y_test)
# As you can seen, the accuracy is ~0.51. In other words, the classifier predicts correctly 51% of the time whether a customer did or din't buy an item.
# Can you improve this baseline logistic regression model by doing better preprocessing and generating new features?
# Also think about my steps! Did it make sense to include the article and customer ids? (And things like that)

0.5087134923350778

In [40]:
# Classification Metrics:
predictions = baseline.predict(X_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.51      0.46      0.48    159054
           1       0.51      0.55      0.53    159475

    accuracy                           0.51    318529
   macro avg       0.51      0.51      0.51    318529
weighted avg       0.51      0.51      0.51    318529



In [41]:
predictions

array([0, 1, 0, ..., 1, 0, 0])

## Assignment: Feature engineering
**TODO:** 
- In groups (of 2-3 students), think about (a few) features that can be engineered (preprocess and generate new features). Divide the work!
- Do these engineered features improve the baseline model?
- Add your thoughts & results to a slide deck for discussion next week (again, 1 slide per person).


## **Preprocess of the data**

In [75]:
# Selecting the features
transactions_processed = transactions[['customer_id', 'age', 'article_id', 'price', 'sales_channel_id', 't_dat', 'ordered', 'colour_group_code']].copy()

In [76]:
# Replacing the nan of age with median
transactions_processed['age'].fillna((transactions_processed['age'].median()))
# Replacing everything left that contains nan with zeros
transactions_processed = transactions_processed.fillna(0)

In [77]:
# Replacing the nan of price with median
transactions_processed['price'].fillna((transactions_processed['price'].median()))

0          0.033881
1          0.033881
2          0.033881
3          0.033881
4          0.033881
             ...   
3185279    0.025407
3185280    0.025407
3185281    0.025407
3185282    0.025407
3185283    0.025407
Name: price, Length: 3185284, dtype: float64

## **The popular items of last week**

In [78]:
# the number of item articles being sold for 1 week. We need to take into account what was order from the customers.
item_date = transactions.loc[transactions["ordered"] == 1][["customer_id", "article_id", "t_dat"]]
item_date['t_dat'] = pd.to_datetime(item_date["t_dat"])
item_date['recently_bought'] = item_date['t_dat'].max()
item_date['days_difference'] = (item_date['recently_bought'] - item_date['t_dat']).dt.days
item_date.head()

,customer_id,article_id,t_dat,recently_bought,days_difference
0,0045c79125b4dc958579f902b49eacd8598f9eeaa12205...,211143037,2018-09-20,2020-09-22,733
1,0045c79125b4dc958579f902b49eacd8598f9eeaa12205...,211143037,2018-09-20,2020-09-22,733
2,c6a3fc64f3c483dab52f69ceabd3c71da48583d5a0160b...,211143037,2018-09-20,2020-09-22,733
3,53f595ec12b425f4bcc972b83954bba9166ff46de4dd9c...,211143037,2018-09-29,2020-09-22,724
4,6a430fdc951a5431e3499c724d5e176914a58c79323383...,211143037,2018-09-26,2020-09-22,727


In [79]:
# the number of items bought in the last week
last_week_items = item_date.loc[item_date["days_difference"] <= 7]
last_week_items = last_week_items["article_id"].value_counts().rename_axis("article_id").reset_index(name="items_bought")
last_week_items

,article_id,items_bought
0,924243001,61
1,923758001,46
2,924243002,41
3,896169005,36
4,762846027,35
...,...,...
5166,903644001,1
5167,892309004,1
5168,747080001,1
5169,669091027,1


## **Preprocessing again with label encoder**

In [80]:
transactions_processed = transactions_processed.merge(last_week_items, how="inner", on='article_id')
customer_encoder = preprocessing.LabelEncoder()
article_encoder = preprocessing.LabelEncoder()
transactions_processed['customer_id'] = customer_encoder.fit_transform(transactions_processed['customer_id'])
transactions_processed['article_id'] = article_encoder.fit_transform(transactions_processed['article_id'])

In [83]:
# Removing the date information for simplicity in prediction
transactions_processed = transactions_processed.drop(['t_dat'], axis=1)

In [84]:
transactions_processed.head()

,customer_id,age,article_id,price,sales_channel_id,ordered,colour_group_code,items_bought
0,69,55.0,2643,0.010153,1,0,61,6
1,48994,48.0,2643,0.010153,1,1,61,6
2,39006,23.0,2643,0.010153,1,1,61,6
3,23342,26.0,2643,0.010153,1,1,61,6
4,27225,54.0,2643,0.009627,1,1,61,6


In [85]:
X_train, X_test, y_train, y_test = train_test_split(transactions_processed.drop('ordered', axis=1), transactions_processed['ordered'], test_size=0.10, random_state=42)
baseline = LogisticRegression(random_state=42)
baseline = baseline.fit(X_train, y_train)
baseline.score(X_test, y_test)

0.7427636613103248

## **One Encoder for the index group name**

In [86]:
# Selecting the features
transactions_processed = transactions[['customer_id', 'age', 'article_id', 'price', 'sales_channel_id', 'ordered', 'index_group_name']].copy()

In [87]:
# Replacing the nan of age with median
transactions_processed['age'].fillna((transactions_processed['age'].median()))
# Replacing everything left that contains nan with zeros
transactions_processed = transactions_processed.fillna(0)
# Replacing the nan of price with median
transactions_processed['price'].fillna((transactions_processed['price'].median()))
customer_encoder = preprocessing.LabelEncoder()
article_encoder = preprocessing.LabelEncoder()
transactions_processed['customer_id'] = customer_encoder.fit_transform(transactions_processed['customer_id'])
transactions_processed['article_id'] = article_encoder.fit_transform(transactions_processed['article_id'])

In [88]:
# Applying the one hot encoder to index group name for 5% of our data
transactions_processed = pd.get_dummies(transactions_processed, columns=['index_group_name'])
transactions_processed.head()

,customer_id,age,article_id,price,sales_channel_id,ordered,index_group_name_Baby/Children,index_group_name_Divided,index_group_name_Ladieswear,index_group_name_Menswear,index_group_name_Sport
0,74,55.0,188,0.033881,2,1,0,0,1,0,0
1,74,55.0,188,0.033881,2,1,0,0,1,0,0
2,53025,24.0,188,0.033881,2,1,0,0,1,0,0
3,22507,57.0,188,0.033881,2,1,0,0,1,0,0
4,28499,22.0,188,0.033881,2,1,0,0,1,0,0


In [89]:
X_train, X_test, y_train, y_test = train_test_split(transactions_processed.drop('ordered', axis=1), transactions_processed['ordered'], test_size=0.10, random_state=42)
baseline = LogisticRegression(random_state=42)
baseline = baseline.fit(X_train, y_train)
baseline.score(X_test, y_test)

0.5134822888967724

## **Combination**

In [90]:
# Selecting the features
transactions_processed = transactions[['customer_id', 'age', 'article_id', 'price', 'sales_channel_id', 't_dat', 'ordered', 'colour_group_code', 'index_group_name']].copy()

In [91]:
transactions_processed = transactions_processed.merge(last_week_items, how="inner", on='article_id')
customer_encoder = preprocessing.LabelEncoder()
article_encoder = preprocessing.LabelEncoder()
transactions_processed['customer_id'] = customer_encoder.fit_transform(transactions_processed['customer_id'])
transactions_processed['article_id'] = article_encoder.fit_transform(transactions_processed['article_id'])

In [ ]:
# Replacing the nan of age with median
transactions_processed['age'].fillna((transactions_processed['age'].median()))
# Replacing everything left that contains nan with zeros
transactions_processed = transactions_processed.fillna(0)
# Replacing the nan of price with median
transactions_processed['price'].fillna((transactions_processed['price'].median()))

In [ ]:
# Applying the one hot encoder to index group name for 5% of our data
transactions_processed = pd.get_dummies(transactions_processed, columns=['index_group_name'])

In [96]:
# Removing the date information for simplicity in prediction
transactions_processed = transactions_processed.drop(['t_dat'], axis=1)
transactions_processed.head()

,customer_id,age,article_id,price,sales_channel_id,ordered,colour_group_code,items_bought,index_group_name_Baby/Children,index_group_name_Divided,index_group_name_Ladieswear,index_group_name_Menswear,index_group_name_Sport
0,69,55.0,2643,0.010153,1,0,61,6,0,0,1,0,0
1,48994,48.0,2643,0.010153,1,1,61,6,0,0,1,0,0
2,39006,23.0,2643,0.010153,1,1,61,6,0,0,1,0,0
3,23342,26.0,2643,0.010153,1,1,61,6,0,0,1,0,0
4,27225,54.0,2643,0.009627,1,1,61,6,0,0,1,0,0


In [97]:
X_train, X_test, y_train, y_test = train_test_split(transactions_processed.drop('ordered', axis=1), transactions_processed['ordered'], test_size=0.10, random_state=42)
baseline = LogisticRegression(random_state=42)
baseline = baseline.fit(X_train, y_train)
baseline.score(X_test, y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.7443732024591678